In [1]:
import csv
import itertools
from time import time
import pickle
import os
import import_ipynb
from hash_tree import Tree, generate_subsets
from timing_wrapper import timeit


importing Jupyter notebook from hash_tree.ipynb
importing Jupyter notebook from timing_wrapper.ipynb


In [2]:
# Important variables
MINSUP = 30 # Minimum support
HASH_DENOMINATOR = 10 # Denominator of the hash function
MIN_CONF = 0.5# Minimum confidence

@timeit
def load_data(path):
    items=[]
    with open(path, 'r') as f:
        reader = csv.reader(f)
        transactions = list(reader)
    for x in transactions:
        items.extend(x)
    items=sorted(set(items))
    return transactions, items

def create_map(items):
    
    map_ = {x:i for i,x in enumerate(items)}
    reverse_map = {i:x for i,x in enumerate(items)}
    return map_, reverse_map

def applymap(transaction, map_):
   
    ret = []
    for item in transaction:
        ret.append(map_[item])
    return ret


In [3]:


#@timeit
def apriori_gen(l_prev):
  
    n = len(l_prev)
    c_curr = []
    for i in range(n):
        for j in range(i+1, n):
            temp_a = l_prev[i]
            temp_b = l_prev[j]
            if temp_a[:-1] == temp_b[:-1]:
                temp_c = []
                temp_c.extend(temp_a)
                temp_c.append(temp_b[-1])
                temp_c=sorted(temp_c)
                c_curr.append(temp_c)
    return c_curr

@timeit
def subset(c_list, transactions):

    candidate_counts={}
    t=Tree(c_list, k=HASH_DENOMINATOR, max_leaf_size=100)
    for transaction in transactions:
        subsets =generate_subsets(transaction, len(c_list[0]))
        for sub in subsets:
            t.check(sub, update=True)
    for candidate in c_list:
        candidate_counts[tuple(candidate)] = t.check(candidate, update=False)
    return candidate_counts



In [4]:
def frequent_itemset_generation(data_path):
    # Uncomment the following lines to load saved pickle file and avoid the extra time required
    # for frequent itemset generation.
    # if 'l_final.pkl' in os.listdir('.'):
    # return pickle.load(open('l_final.pkl', 'rb'))
    transactions, items = load_data(data_path)
    map_, reverse_map = create_map(items)
    pickle.dump(reverse_map, open('reverse_map.pkl', 'wb+'))
    one_itemset = [[itemset] for itemset in items]
    items_mapped = [applymap(itemset, map_) for itemset in one_itemset]
    transactions_mapped = [applymap(transaction, map_) for transaction in transactions]
    temp_l_current = subset(items_mapped, transactions_mapped)
    l_current={}
    for t in temp_l_current.keys():
        if temp_l_current[t] > MINSUP:
            l_current[tuple(t)] = temp_l_current[t]
    L_final = []
    L_final.append(l_current)

   
    while(len(l_current)):
        c_current = apriori_gen(list(l_current.keys()))
        if len(c_current):
            C_t = subset(c_current, transactions_mapped)
            l_current = {}
            for c in C_t.keys():
                if C_t[c] > MINSUP:
                    l_current[tuple(sorted(c))] = C_t[c]
            if len(l_current):
                L_final.append(l_current)
        else:
            break
    pickle.dump(L_final, open('l_final.pkl', 'wb+'))
    return L_final






In [5]:
def generate_rules(frequent_items):
    rules=[]
    for k_itemset in frequent_items:
        k=len(list(k_itemset.keys())[0])
        if k==1: # No rules can be generated using 1 itemsets
            continue
        for itemset, support in k_itemset.items():
            H_curr=[[x] for x in itemset]
            to_remove=[]
            for h in H_curr:
                X=tuple(sorted(set(itemset)-set(h)))
                Y=tuple(sorted(h))
                confidence = support / (frequent_items[k-2][X])
                if confidence > MIN_CONF:
                    rule=[]
                    rule.append(X)
                    rule.append(Y)
                    rules.append({tuple(rule):confidence})
                else:
                    to_remove.append(h)

            H_curr=[x for x in H_curr if x not in to_remove]

            for m in range(1,k-1):
                if k > m+1:
                    H_next=apriori_gen(H_curr)
                    to_remove=[]
                    for h in H_next:
                        X=tuple(sorted(set(itemset)-set(h)))
                        Y=tuple(sorted(h))
                        confidence = support / (frequent_items[k-m-2][X])
                        if confidence>MIN_CONF:
                            rule=[]
                            rule.append(X)
                            rule.append(Y)
                            rules.append({tuple(rule):confidence})
                        else:
                            to_remove.append(h)
                    H_next=[x for x in H_next if x not in to_remove]
                    H_curr=H_next
                else:
                    break
    return rules

In [6]:
def display_rules(rules, frequent_items, write=False):
    reverse_map=pickle.load(open('reverse_map.pkl', 'rb'))
    bad_chars="[]''"
    with open('outputs/association_rules.txt', 'w+') as f:
        for rule in rules:
            X, Y=list(rule.keys())[0]
            precedent_support_count, antecedent_support_count=(frequent_items[len(X)-1][X], frequent_items[len(Y)-1][Y])
            confidence=list(rule.values())[0]
            print(str([reverse_map[x] for x in X]).strip(bad_chars).replace("'", '')+'('+str(precedent_support_count)+')'+' ---> '+str([reverse_map[y] for y in Y]).strip(bad_chars).replace("'", '') +'('+str(antecedent_support_count)+')' + ' - conf('+ str(confidence)+ ')')
            f.write(str([reverse_map[x] for x in X]).strip(bad_chars).replace("'", '')+'('+str(precedent_support_count)+')'+' ---> '+str([reverse_map[y] for y in Y]).strip(bad_chars).replace("'", '') +'('+str(antecedent_support_count)+')' + ' - conf('+ str(confidence)+ ')'+'\n')

    with open('outputs/frequent_itemsets.txt', 'w+') as f:
        for k_itemset in frequent_items:
            for itemset, support in k_itemset.items():
                f.write(str([reverse_map[x] for x in itemset]).strip(bad_chars).replace("'", '')+' ('+str(support)+')'+'\n')


In [7]:
if __name__=='__main__':
    data_path = 'datasets/dishes.csv'
    print(data_path)
    frequent_items = frequent_itemset_generation(data_path)
    rules = generate_rules(frequent_items)
    display_rules(rules, frequent_items, write=True)
    no_itemsets=0
    for x in frequent_items:
        no_itemsets+=len(x)
print('No of rules:',len(rules), 'No of itemsets:', no_itemsets)

datasets/dishes.csv
load_data took 0.031502723693847656 seconds.
subset took 0.5302462577819824 seconds.
subset took 32.56483864784241 seconds.
subset took 1.7055048942565918 seconds.
subset took 0.35089588165283203 seconds.
 Beef Burger(51) ---> Burgers(541) - conf(0.7647058823529411)
 Veg Burger(69) --->  Burgers(775) - conf(0.5072463768115942)
 Chicken Boneless Biryani(70) --->  Vegetable Biryani(268) - conf(0.5428571428571428)
 Chicken Burger(89) --->  Pasta(1265) - conf(0.5393258426966292)
Shawarma(82) --->  Chicken Grill(204) - conf(0.524390243902439)
 Thukpa(51) --->  Chicken Momo(80) - conf(0.6078431372549019)
 Chicken Roll(80) ---> Rolls(204) - conf(0.575)
 Craft Beer(88) --->  Cocktails(823) - conf(0.6818181818181818)
 Jumbo Prawns(52) --->  Cocktails(823) - conf(0.7115384615384616)
 Sangria(120) --->  Cocktails(823) - conf(0.6083333333333333)
Beer(151) --->  Cocktails(823) - conf(0.5761589403973509)
 Kesari Bath(47) --->  Filter Coffee(190) - conf(0.6595744680851063)
 Wings(